In [2]:
%matplotlib inline

from astropy.io import fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import random
import math
from scipy import ndimage

ppp = 3500*math.tan(1.2/3600 * np.pi/180) # parsec per pixel length
print(ppp)
pc_to_cm_area = 9.521*1e36 # parsec^2 to cm^2
solar_mass = 1.9885e30 # 1 Solar mass in kg

plt.style.use({"image.origin": "lower", "image.interpolation": "nearest"})

FITS_PATH = '../data/fits/PROMISE-Q1-8micron-filled-v0_3.fits'
# FITS_PATH = '../cnn/fits_for_annotation/box(50316.355, 6033.3671, 234.3168, 178.3296, 0.0).fits'
# Open img using memmap to not load the whole image into memory
imageHDU = fits.open(FITS_PATH, memmap=True, mode='denywrite')

# read catalog.csv
catalog = pd.read_csv('../data/cnn_negative_examples_merged_masks.csv', index_col=False, dtype={"x_center": np.int32, "y_center": np.int32, "approx_size": np.int32, "box_x1": np.int32, "box_x2": np.int32, "box_y1": np.int32, "box_y2": np.int32, "mask_file": str})


mass_column = []
size_pc_column = []
for i in range(len(catalog)):
    row = catalog.iloc[i]
    padding = 80
    data = np.array(imageHDU[0].data[row["box_y1"]-padding:row["box_y2"]+padding, row["box_x1"]-padding:row["box_x2"]+padding], dtype=np.float32) # type: ignore

    data_box_Ioff = np.array(imageHDU[0].data[row["box_y1"]:row["box_y2"], row["box_x1"]: row["box_x2"]], dtype=np.float32)
    data_box_Iobs = np.array(imageHDU[0].data[row["box_y1"]:row["box_y2"], row["box_x1"]: row["box_x2"]], dtype=np.float32)

    mask = np.load("../data/masks/"+row["mask_file"])
    rows, cols = mask.shape
    try:
        for r in range(rows):
            for c in range(cols):
                if mask[r,c] == True:
                    data_box_Ioff[r,c] = None
                else:
                    data_box_Iobs[r,c] = None

        flat_data_box_Iobs = np.sort(data_box_Iobs.flatten())
        I_fg = np.nanmean(flat_data_box_Iobs[:10])
        I_obs = np.nanmean(flat_data_box_Iobs)
        I_off = np.nanmean(data_box_Ioff.flatten())

        tau8 = -np.log((I_obs - I_fg)/(I_off - I_fg))
        kappa8 = 7.5 #cm^2 /g
        sigma8 = tau8/kappa8
        approx_size_of_cloud = row[2]
        area_pc = approx_size_of_cloud*ppp**2
        fat_sun = area_pc*pc_to_cm_area*sigma8/1000
        IRDC_SM = fat_sun/solar_mass
        size_pc_column.append(area_pc)
        mass_column.append(IRDC_SM)
    except:
        print(f"Error: index {i} caused an indexing issue. Setting mass to NaN.")
        mass_column.append(np.nan)


catalog['Mass [M_\u2609]'] = mass_column
catalog['Size [pc]'] = size_pc_column

# print(f'approximate size of cloud: {approx_size_of_cloud}')
# print(f'tau_8: {tau8} sigma_8: {sigma8}')
# print(f'approximate mass of cloud: {fat_sun} kg or {IRDC_SM} solar masses')





#---------------------------------------------------- observer pixel corresponds to an angle IN HEADER: [CD2_2], distance d=3,5 kpc------------100s solar masses smallers clouds???--------biggest 10^5 solar masses-------------------------------------------------------------------------
# for smaller clouds the fg is harder to find. We can't just pick the darkest pixel. 
# Cosmic conspiracy.


0.02036217460683024


C:\Users\mull_\AppData\Local\Temp\ipykernel_2228\1349128689.py:56: RuntimeWarning: divide by zero encountered in log
  tau8 = -np.log((I_obs - I_fg)/(I_off - I_fg))
C:\Users\mull_\AppData\Local\Temp\ipykernel_2228\1349128689.py:54: RuntimeWarning: Mean of empty slice
  I_off = np.nanmean(data_box_Ioff.flatten())
C:\Users\mull_\AppData\Local\Temp\ipykernel_2228\1349128689.py:56: RuntimeWarning: invalid value encountered in log
  tau8 = -np.log((I_obs - I_fg)/(I_off - I_fg))


In [6]:
#save catalog as new csv
catalog.to_csv('../data/cnn_negative_examples_merged_masks_mass.csv', index=False)

In [5]:
print(catalog["Mass [M_\u2609]"])

0          9.768219
1         64.163742
2         21.564314
3         33.584039
4        720.131252
            ...    
17709     39.473373
17710     25.319235
17711    230.278360
17712    830.238005
17713     65.894852
Name: Mass [M_☉], Length: 17714, dtype: float64
